In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("../data/past/clean/19_20.csv")
home_df = ((df[['Date','HomeTeam','AwayTeam','FTHG','FTAG']]).rename(columns={'HomeTeam':'Team','AwayTeam':'Opponent','FTHG':'GoalsScored','FTAG':'GoalsAgainst'})).assign(IsHome='Y')
away_df = ((df[['Date','AwayTeam','HomeTeam','FTAG','FTHG']]).rename(columns={'AwayTeam':'Team','HomeTeam':'Opponent','FTAG':'GoalsScored','FTHG':'GoalsAgainst'})).assign(IsHome='N')
real_df = pd.concat([home_df,away_df])

In [5]:
hometeam_avg = home_df.groupby('Team').agg(HomeAvgScored=('GoalsScored','mean'),HomeAvgAgainst=('GoalsAgainst','mean'),Matches=('Date','count'))
awayteam_avg = away_df.groupby('Team').agg(AwayAvgScored=('GoalsScored','mean'),AwayAvgAgainst=('GoalsAgainst','mean'),Matches=('Date','count'))
avg = (pd.merge(hometeam_avg,awayteam_avg,left_index=True, right_index=True)).rename(columns={'Matches_x':'HomeMatches','Matches_y':'AwayMatches'})
avg['AvgScored'] = (avg['HomeAvgScored'])*(avg['HomeMatches']/(avg['HomeMatches']+avg['AwayMatches'])) + (avg['AwayAvgScored'])*(avg['AwayMatches']/(avg['HomeMatches']+avg['AwayMatches']))
avg['AvgAgainst'] = (avg['HomeAvgAgainst'])*(avg['HomeMatches']/(avg['HomeMatches']+avg['AwayMatches'])) + (avg['AwayAvgAgainst'])*(avg['AwayMatches']/(avg['HomeMatches']+avg['AwayMatches']))
avg['HomeFactor'] = avg['HomeAvgScored']/avg['AwayAvgScored']
avg['Matches'] = avg['HomeMatches'] + avg['AwayMatches']
avg[['AvgScored','AvgAgainst','HomeFactor','Matches']]

,AvgScored,AvgAgainst,HomeFactor,Matches
Team,,,,
Aves,0.705882,2.000000,1.000000,34
Belenenses,0.794118,1.588235,0.928571,34
Benfica,2.088235,0.764706,1.151515,34
Boavista,0.823529,1.147059,1.153846,34
Famalicao,1.558824,1.500000,1.120000,34
Gil Vicente,1.176471,1.294118,1.857143,34
Guimaraes,1.558824,1.117647,1.038462,34
Maritimo,1.000000,1.235294,2.400000,34
Moreirense,1.235294,1.294118,1.210526,34


In [34]:

df2 = pd.read_csv("../data/past/clean/20_21.csv")
teams = avg.index.unique()
lambda_forces = pd.DataFrame({'Team':teams,'LambdaAtt':0,'LambdaDef':0,'HomeFactor':0})

league_avg_scored = avg['AvgScored'].mean()
league_avg_against = avg['AvgAgainst'].mean()
league_avg_goals = league_avg_scored + league_avg_against

    #lambda(Home) = lambdaAtt(Home) * lambdaDef(Away) * HomeFactor(Home)
    #lambda(Away) = lambdaAtt(Home) * lambdaDef(Away) * (2 - HomeFactor(Away))
lambda_forces['LambdaAtt'] = lambda_forces['Team'].map(lambda t: avg.loc[t,'AvgScored'] / league_avg_scored)
lambda_forces['LambdaDef'] = lambda_forces['Team'].map(lambda t: avg.loc[t,'AvgAgainst'] / league_avg_against)
lambda_forces['HomeFactor'] = lambda_forces['Team'].map(lambda t: avg.loc[t,'HomeFactor'])
lambda_forces.loc[lambda_forces['Team'] == 'Benfica']
lf = lambda_forces
lf[['LambdaAtt']] * 0.75

,LambdaAtt
0,0.424640
1,0.477720
2,1.256225
3,0.495413
4,0.937746
5,0.707733
6,0.937746
7,0.601573
8,0.743119
9,0.636959


In [7]:
    lf_home = lf.loc[lf['Team'] == 'Benfica']
    lf_away = lf.loc[lf['Team'] == 'Tondela']

    #equivalente a LambdaA
    #lambda(Home) = lambdaAtt(Home) * lambdaDef(Away) * HomeFactor(Home) 
    #lambda(Away) = lambdaAtt(Home) * lambdaDef(Away) * (2 - HomeFactor(Away))
    lambda_home = lf_home['LambdaAtt'].iloc[0] * lf_away['LambdaDef'].iloc[0] * lf_home['HomeFactor'].iloc[0]
    lambda_away = lf_away['LambdaAtt'].iloc[0] * lf_home['LambdaDef'].iloc[0] * (2 - lf_away['HomeFactor'].iloc[0])
    round(lambda_away,2)

np.float64(0.58)

In [8]:
from scipy.stats import poisson
import numpy as np

lambda_home = 2
lambda_away = 0.5
k = np.arange(0, 10)
prob_home = poisson.pmf(k, lambda_home)
prob_away = poisson.pmf(k, lambda_away)
goals_home = goals_away = 0
max_prob_home = max_prob_away = 0
for i in range(0,10):
    home = prob_home[i]
    away = prob_away[i]
    print(f"{i}: {home * 100}% for HOME TO SCORE // {away * 100}% for AWAY TO SCORE")
    if (home >= max_prob_home):
        max_prob_home = home
        goals_home = i
    if (away >= max_prob_away):
        max_prob_away = away
        goals_away = i
print(f"{goals_home} ({round(max_prob_home * 100,1)}%) : {goals_away} ({round(max_prob_away * 100,1)}%)")

0: 13.53352832366127% for HOME TO SCORE // 60.653065971263345% for AWAY TO SCORE
1: 27.06705664732254% for HOME TO SCORE // 30.326532985631673% for AWAY TO SCORE
2: 27.06705664732254% for HOME TO SCORE // 7.581633246407919% for AWAY TO SCORE
3: 18.044704431548357% for HOME TO SCORE // 1.2636055410679865% for AWAY TO SCORE
4: 9.022352215774179% for HOME TO SCORE // 0.15795069263349829% for AWAY TO SCORE
5: 3.6089408863096724% for HOME TO SCORE // 0.01579506926334984% for AWAY TO SCORE
6: 1.2029802954365565% for HOME TO SCORE // 0.0013162557719458203% for AWAY TO SCORE
7: 0.34370865583901616% for HOME TO SCORE // 9.401826942470143e-05% for AWAY TO SCORE
8: 0.08592716395975401% for HOME TO SCORE // 5.8761418390438375e-06% for AWAY TO SCORE
9: 0.019094925324389823% for HOME TO SCORE // 3.2645232439132406e-07% for AWAY TO SCORE
2 (27.1%) : 0 (60.7%)


In [9]:
def predictGame(game,lf):
    # lf vai ser um mix do velho com um novo
    # i.e: criar outro lambdaForce com o DF das avg até ao momento do jogo desta season
    # atualizar LAMBDA com EWMA/rolling update (ultimo jogo tem X peso e o historico tem outro X)

    #λ_att_new_A = α * gols_marcados_no_jogo + (1 - α) * λ_att_old_A
    #λ_def_new_B = α * gols_sofridos_no_jogo + (1 - α) * λ_def_old_B
    # nem será preciso criar o lambda a partir das avg'?

    home_team = game.loc['HomeTeam']
    away_team = game.loc['AwayTeam']
    lf_home = lf.loc[lf['Team'] == home_team]
    lf_away = lf.loc[lf['Team'] == away_team]

    if (lf_home.empty):
        # Use adjusted means in teams that just got promoted
        lambda_home_att = (lf['LambdaAtt'].mean()) * 0.9
        lambda_home_def = (lf['LambdaDef'].mean()) * 1.1
        home_factor = lf['HomeFactor'].mean()
    else:
        lambda_home_att = lf_home['LambdaAtt'].iloc[0]
        lambda_home_def = lf_home['LambdaDef'].iloc[0]
        home_factor = lf_home['HomeFactor'].iloc[0]

    if (lf_away.empty):
        # Use adjusted means in teams that just got promoted
        lambda_away_att = (lf['LambdaAtt'].mean()) * 0.9
        lambda_away_def = (lf['LambdaDef'].mean()) * 1.1
        away_factor = lf['HomeFactor'].mean()
    else:
        lambda_away_att = lf_away['LambdaAtt'].iloc[0]
        lambda_away_def = lf_away['LambdaDef'].iloc[0]
        away_factor = lf_away['HomeFactor'].iloc[0]

    #equivalente a LambdaA
    #lambda(Home) = lambdaAtt(Home) * lambdaDef(Away) * HomeFactor(Home) 
    #lambda(Away) = lambdaAtt(Home) * lambdaDef(Away) * (2 - HomeFactor(Away))
    lambda_home = lambda_home_att * lambda_away_def * home_factor
    lambda_away = lambda_away_att * lambda_home_def * (2 - away_factor)

    # At max 9 goals (in order to make an accurate enough prediction)
    k = np.arange(0, 10)
    prob_home = poisson.pmf(k, lambda_home)
    prob_away = poisson.pmf(k, lambda_away)

    goals_home = goals_away = 0
    max_prob_home = max_prob_away = 0
    for i in range(0,10):
        home = prob_home[i]
        away = prob_away[i]
        #print(f"{i}: {home * 100}% for HOME TO SCORE // {away * 100}% for AWAY TO SCORE")
        if (home >= max_prob_home):
            max_prob_home = home
            goals_home = i
        if (away >= max_prob_away):
            max_prob_away = away
            goals_away = i
    #print(f"{goals_home} ({round(max_prob_home * 100,1)}%) : {goals_away} ({round(max_prob_away * 100,1)}%)")
    return goals_home,goals_away

In [22]:
    calendar = df2[['Date','HomeTeam','AwayTeam']].copy()

    calendar['PredHomeGoals'] = 0
    calendar['PredAwayGoals'] = 0
    calendar['RealHomeGoals'] = 0
    calendar['RealAwayGoals'] = 0

    for id in calendar.index:
        game = calendar.loc[id]
        goals_home,goals_away = predictGame(game,lf)
        calendar.loc[id, 'PredHomeGoals'] = goals_home
        calendar.loc[id, 'PredAwayGoals'] = goals_away
        calendar.loc[id, 'RealHomeGoals'] = df.loc[id,'FTHG']
        calendar.loc[id, 'RealAwayGoals'] = df.loc[id,'FTAG']
        game = calendar.loc[id]
        print(game)

Date             18/09/2020
HomeTeam          Famalicao
AwayTeam            Benfica
PredHomeGoals             0
PredAwayGoals             1
RealHomeGoals             0
RealAwayGoals             0
Name: 0, dtype: object
Date             18/09/2020
HomeTeam          Guimaraes
AwayTeam         Belenenses
PredHomeGoals             1
PredAwayGoals             0
RealHomeGoals             0
RealAwayGoals             2
Name: 1, dtype: object
Date             19/09/2020
HomeTeam           Nacional
AwayTeam           Boavista
PredHomeGoals             1
PredAwayGoals             0
RealHomeGoals             2
RealAwayGoals             1
Name: 2, dtype: object
Date             19/09/2020
HomeTeam              Porto
AwayTeam           Sp Braga
PredHomeGoals             2
PredAwayGoals             0
RealHomeGoals             5
RealAwayGoals             0
Name: 3, dtype: object
Date              20/09/2020
HomeTeam         Santa Clara
AwayTeam            Maritimo
PredHomeGoals              0
PredAway